In [25]:
# -*- coding: utf-8 -*-
import nltk,re
from nltk import word_tokenize 
import itertools
import numpy as np
import time
import sys
import operator
import io
import os
import array
from six.moves import cPickle as pickle
from datetime import datetime
import codecs

In [77]:
path='/user/fnoorala/home/Desktop/SMILK/InformationExtraction/data/annotated_cosmetic_corpus.xml'
pathtest='/user/fnoorala/home/Desktop/SMILK/InformationExtraction/data/test.xml'
SENTENCE_START_TOKEN = "SENTENCE_START"
SENTENCE_END_TOKEN = "SENTENCE_END"
UNKNOWN_TOKEN = "<UNK>"
word_to_index = []
index_to_word = []
vocabulary_size=8000




def readfiles(filename):
     with codecs.open(filename, encoding='utf-8') as f:
        lines = [line.rstrip('\n') for line in open(filename) 
                 if (len(line.strip())>0)]
        return lines

In [78]:
def extractTags(mystr):
        tags=[ match.group(1) for match in re.finditer('<(?P<tag>\w+)>(.*?)</(?P=tag)>',mystr)]
        return tags

In [79]:
def replacement(x):
    ne= x.group(1)
    tokens=nltk.word_tokenize(x.group(2))
    rep=""
    for j in range(len(tokens)):
        rep+=("B+"+ne if j==0 else "I+"+ne)+" "
    return rep.strip()
def clearTags(x):
    tokens=nltk.word_tokenize(x.group(2))
    rep=""
    for j in range(len(tokens)):
        rep+=(tokens[j] if j==0 else tokens[j])+" "
    return rep.strip()
    


In [85]:
def transformation(filename):
    dataset=readfiles(filename)
    texts=[]
    reltags=[]
    entitytags=[]
    
    for k in range(0,len(dataset)):
        
        result_d = {}
        text=dataset[k].decode("utf-8")
        rep1=re.sub('<(?P<tag>\w+)>(.*?)</(?P=tag)>',clearTags,text)
        rep2=re.sub('<(?P<tag>\w+)>(.*?)</(?P=tag)>',replacement,text)
        rg = re.compile('[BI]\+(?:[A-Za-z][a-z0-9_]*)')
        t=nltk.word_tokenize(rep1)
        tokens=nltk.word_tokenize(rep2)
        
        words= [x.lower() for x in t]
        lables=[ x if rg.match(x) is not None else 'O' for x in tokens ]
        texts.append(words)
        reltags.append(lables)
     
    return texts,reltags

In [ ]:
def dataset(path):
    
    train=''
    
    train_set, train_labels=transformation(train)
      
    
    #sentences = ['%s %s %s' % (SENTENCE_START_TOKEN, x, SENTENCE_END_TOKEN) for x in sentences]
    #labels=[[SENTENCE_START_TOKEN]+l+[SENTENCE_END_TOKEN] for l in labels]

    # Tokenize the sentences into words
    
    train_tokenized_sentences = [nltk.word_tokenize(sent) for sent in train_set]
    
    #we converted sequences of numbers with the string DIGIT i.e. 1984
    #is converted to DIGITDIGITDIGITDIGIT.
  
    train_tokenized_sentences=[[re.sub(r"\d+",lambda m:convertToDigit(m.group()),w) for w in t ] for t in train_tokenized_sentences]
    

    
    # Count the word frequencies
    word_freq = nltk.FreqDist(itertools.chain(*train_tokenized_sentences))
    
    print("Found %d unique words tokens." % len(word_freq.items()))

    # Get the most common words and build index_to_word and word_to_index vectors
    vocab = sorted(word_freq.items(), key=lambda x: (x[1], x[0]), reverse=True)[:vocabulary_size-2]

    #we deal with unseen words in the test set by marking any words with only one 
    #single occurrence in the training set as <UNK>
    vocab = [x for x in vocab if not x[1]<2]

    #print("Using vocabulary size %d." % vocabulary_size)
    print("The least frequent word in our vocabulary is '%s' and appeared %d times." % (vocab[-1][0], vocab[-1][1]))
    sorted_vocab = sorted(vocab, key=operator.itemgetter(1))
   
    index_to_word = ["<MASK/>", UNKNOWN_TOKEN] + [x[0] for x in sorted_vocab]
    word_to_index = dict([(w, i) for i, w in enumerate(index_to_word)])

 
    # Replace all words not in our vocabulary with the unknown token
    for i, sent in enumerate(train_tokenized_sentences):
        train_tokenized_sentences[i] = [w if w in word_to_index else UNKNOWN_TOKEN for w in sent]
    for i, sent in enumerate(test_tokenized_sentences):
        test_tokenized_sentences[i] = [w if w in word_to_index else UNKNOWN_TOKEN for w in sent]

    lables_1=[item for sublist in train_labels_1 for item in sublist]
    lables_2=[item for sublist in train_labels_2 for item in sublist]
    
    
    
    index_to_label_1=['<MASK/>']+[i for i in np.unique(lables_1)]
    index_to_label_2=['<MASK/>']+[i for i in np.unique(lables_2)]
    
    label_to_index_1 = dict([(w, i) for i, w in enumerate(index_to_label_1)])
    label_to_index_2 = dict([(w, i) for i, w in enumerate(index_to_label_2)])
    
    dicts={'words2idx':word_to_index,'labels2idx_1':label_to_index_1,'labels2idx_2':label_to_index_2}
    # Create the training dataset and test dataset
    X_train = np.asarray([[word_to_index[w] for w in sent] for sent in train_tokenized_sentences])
    y1_train = np.asarray([[label_to_index_1[w] for w in sent] for sent in train_labels_1])
    y2_train = np.asarray([[label_to_index_2[w] for w in sent] for sent in train_labels_2])
      
    X_test = np.asarray([[word_to_index[w] for w in sent] for sent in test_tokenized_sentences])
    y1_test = np.asarray([[label_to_index_1[w] for w in sent] for sent in test_labels_1])
    y2_test = np.asarray([[label_to_index_2[w] for w in sent] for sent in test_labels_2])
                        
    
    semeval={'train_dataset':X_train,'train_labels_1':y1_train,'train_labels_2':y2_train,'test_dataset':X_test,
            'test_labels_1':y1_test,'test_labels_2':y2_test,'dicts':dicts}
    return semeval
#save the train, test and dictionary in a serialized object
pickle.dump( dataset(path), open( "semeval.pkl", "wb" ) )